Authors: Roberta, Dora, Zigfrid  
dependecies: Python 3.6.4+

# TF IDF Report

## Table of Content 

1. Map Reduce Algorithm: a description of the _adopted solution_

    1a. Other designed algorithms plus related global comments/description including comments to the code 

3. Experimental analysis, concerning in particular scalability 

    2a. Comments about the experimental analysis outlining weak and strong points of the algorithms.

4. an appendix including all the code

# 1. MapReduce algorithm
calculate TF-IDF scores given an input set of documents. Provide both a MapReduce Python Hadoop streaming and a Spark implementation. 
## 1.a. Adopted Version

### Python Hadoop

### Spark

## 1.b. Other designed algorithms

### Python Hadoop

### Spark


# 2. Experimental analysis
compare performances of the two implementation. To test scalability consider 5 document sets of increasing sizes. For instance, size can double from a set to another including  comments about the experimental analysis outlining weak and strong points of the algorithms.

Text source ideas - project guttenberg: https://www.gutenberg.org/
movie transcripts - https://www.imsdb.com/TV/Futurama.html


### Practical Instructions to run tests on the cluster

Collect speed information here: https://docs.google.com/spreadsheets/d/1woubFQECRr-0GMBUtKGj6vq0i8aybqAdAYiR-eaTaOY/edit?usp=sharing

1. Create text
* create input directory on /user/hadoop local file system 
* in /input, wget python script and execute the text_generator function to create documents needed, specify params as described below. if you already created soem files, remember to change the startnr.
* afterwards, move only the .txt to the hdfs hadoop input directory 

2. Download the scripts mapper 1 - reducer 3, and the .bash script
 + give permission 

After one job is done, remove all input and output directories, and create a new input directory on hdfs to be filled with the new text documents.

# 3. Code Appendix